In [330]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from datetime import datetime, timedelta

In [331]:
data = pd.read_csv('module_4-project_4-flights.csv')

In [332]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127 entries, 0 to 126
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   flight_id            127 non-null    int64  
 1   departure_city       127 non-null    object 
 2   departure_latitude   127 non-null    float64
 3   departure_longitude  127 non-null    float64
 4   arrival_city         127 non-null    object 
 5   arrival_latitude     127 non-null    float64
 6   arrival_longitude    127 non-null    float64
 7   scheduled_departure  127 non-null    object 
 8   scheduled_arrival    127 non-null    object 
 9   actual_departure     127 non-null    object 
 10  actual_arrival       127 non-null    object 
 11  status               127 non-null    object 
 12  aircraft_code        127 non-null    object 
 13  model                127 non-null    object 
 14  range                127 non-null    int64  
 15  aircraft_seats       127 non-null    int

Посмотрим, сколько у нас зимних рейсов из Анапы.

In [333]:
data.flight_id.nunique()

127

Узнаем, количество мест в самолете и количество мест, которые были заняты на этих рейсах

In [334]:
fullness = data[['flight_seats','aircraft_seats']]
fullness['per'] = round((fullness['flight_seats'] / fullness['aircraft_seats'])*100,2)
fullness[fullness['per'] < 80]

<ipython-input-334-2dafc7499065>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fullness['per'] = round((fullness['flight_seats'] / fullness['aircraft_seats'])*100,2)


,flight_seats,aircraft_seats,per
2,97,130,74.62
12,99,130,76.15
15,100,130,76.92
21,99,130,76.15
38,97,130,74.62
39,103,130,79.23
54,100,130,76.92
59,0,130,0.00
60,0,130,0.00
61,0,130,0.00


# Из предыдущего теста мы выяснили, что есть рейсы, на которых не летели пассажиры, поэтому от них можно избавиться в связи с невостребованностью направления (все рейсы в Новокузнецк) и использовать освободившиеся самолеты для выбора востребованного и прибыльного направления

In [388]:
a = data[data['flight_seats'] == 0].flight_id.unique()
print(f'Авиарейсы в Новокузнецк, которые можно исключить для перераспределения в востребованные направления: {list(a)} ')

Авиарейсы в Новокузнецк, которые можно исключить для перераспределения в востребованные направления: [136511, 136513, 136514, 136523, 136540, 136544, 136546, 136560, 136567] 


# Кроме того, проверим, нужны ли все рейсы, которые совершаются в другие города (Москва, Белгород)

In [336]:
Moscow_aircrafts = data[(data['flight_seats'] != 0) & (data['arrival_city'] == 'Moscow')].aircraft_seats.sum()
Moscow_fullness = data[(data['flight_seats'] != 0) & (data['arrival_city'] == 'Moscow')].flight_seats.sum()
print(f"Общее количество мест в самолетах, которые летают в Москву: {Moscow_aircrafts} ")
print(f"Общее количество мест, занятых пассажирами, в самолетах, которые летают в Москву: {Moscow_fullness} ")
print(f"Общий показатель заполненности на рейсы в Москва: {round(Moscow_fullness/Moscow_aircrafts*100,2)} %")

Общее количество мест в самолетах, которые летают в Москву: 7670 
Общее количество мест, занятых пассажирами, в самолетах, которые летают в Москву: 6674 
Общий показатель заполненности на рейсы в Москва: 87.01 %


In [337]:
Belgorod_aircrafts = data[(data['flight_seats'] != 0) & (data['arrival_city'] == 'Belgorod')].aircraft_seats.sum()
Belgorod_fullness = data[(data['flight_seats'] != 0) & (data['arrival_city'] == 'Belgorod')].flight_seats.sum()
print(f"Общее количество мест в самолетах, которые летают в Белгород: {Belgorod_aircrafts} ")
print(f"Общее количество мест, занятых пассажирами, в самолетах, которые летают в Белгород: {Belgorod_fullness} ")
print(f"Общий показатель заполненности на рейсы в Белгород: {round(Belgorod_fullness/Belgorod_aircrafts*100,2)} %")

Общее количество мест в самолетах, которые летают в Белгород: 5723 
Общее количество мест, занятых пассажирами, в самолетах, которые летают в Белгород: 5321 
Общий показатель заполненности на рейсы в Белгород: 92.98 %


# Создадим новые признаки для подсчета прибыльности рейсов

In [339]:
# Создадим признак с дистанцией с помощью координат из нашего датасета 
distance_bw_cities= {}
for i in data['arrival_city'].unique():
    
    a = data['departure_latitude'][data['departure_city'] == 'Anapa'][0]
    b = data['departure_longitude'][data['departure_city'] == 'Anapa'][0]
    c = data['arrival_latitude'][data['arrival_city'] == i].iloc[0]
    d = data['arrival_longitude'][data['arrival_city'] == i].iloc[0]
    
    distance_bw_cities[i] = geodesic((a,b),(c,d)).km
     


data['flight_distance_km'] = data['arrival_city'].map(distance_bw_cities)

In [340]:
data['actual_departure'] = data['actual_departure'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ'))
data['actual_arrival'] = data['actual_arrival'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ'))

In [341]:
data['time_range'] = data['actual_arrival'] - data['actual_departure']
data['time_range'] = data['time_range'].apply(lambda x: (x.total_seconds())/60/60)

In [342]:
# 
print(data['model'].unique())
#Берем данные по стоимости за 1 кг керосина в аэропорту Анапы 
#на конец 2017 года с сайта Федерального агенства воздушного транспорта (БЕЗ НДС)
data['fuel_for_kg'] = 47.101
#Проведем рассчет, сколько тратится топлива за один рейс
fuel_for_hour = {'Boeing 737-300':2400,'Sukhoi Superjet-100':1800}
data['fuel_for_hour'] = data['model'].map(fuel_for_hour)

data['fuel_costs'] = round(data['fuel_for_kg'] * data['fuel_for_hour'] * data['time_range'],2)

['Boeing 737-300' 'Sukhoi Superjet-100']


In [343]:
# Из информации полученной из интернета оба самолета летают с экипажем 2 летчика и 2 бортпроводника
# из полученных из интернета данных высчитаем примерные затраты на экипаж
staff_costs = (150000 / 75 * 2) + (63000 / 75 * 2)
data['staff_costs'] = round(data['time_range'] * staff_costs,2)

In [344]:
#не представляется возможным выяснить затраты 
#на техническое и иное обслуживание самолета, потому что нет необходимой информации
#при наличии бухгалтерских регистров можно было бы выяснить затраты на обслуживание
#и разделить на количество осществленных рейсов самолетом за определенный период

In [345]:
#Налоги и сборы
#Косвенные налоги (входящий ндс на топливо (в 2017 году - 18%), будем считать,что акцизы (2300 р за 1 тонну) включены)
# не будем проводить расчет остального ндс, потому что у нас нет необходимой информации
data['ind_taxes'] = round(data['fuel_costs'] * 0.18,2)
#Работодатель платит страховые взносы в размере 34% от заработной платы работника 
#(однако при превышении определенного предела размер может уменьшаться, но мы не будем это учитывать в нашей задаче)
data['fees'] = round(data['staff_costs'] * 0.34,2)
#После расчета прибыли по истечении финансового года уплачивается налог на прибыль организаций (20%)
data['inc_tax'] = (data['total_amount'] - data['fuel_costs'] - data['staff_costs'] - data['ind_taxes'] - data['fees']) * 0.2

In [346]:
data['income'] = round((data['total_amount'] - data['fuel_costs'] - data['staff_costs'] - data['ind_taxes'] - data['fees'] - data['inc_tax']),2)

# Проверим наименее прибыльные рейсы. Все рейсы относятся к Белгороду, однако без знания методологии расчета себестоимости каждого рейса, тяжело оценить прибыльность рейса.

In [384]:
smp = data[data['income'] > 0]
qt25 = np.quantile(np.array(smp['income']),0.10)
smp[smp['income'] < qt25]

,flight_id,departure_city,departure_latitude,departure_longitude,arrival_city,arrival_latitude,arrival_longitude,scheduled_departure,scheduled_arrival,actual_departure,...,flight_distance_km,time_range,fuel_for_kg,fuel_for_hour,fuel_costs,staff_costs,ind_taxes,fees,inc_tax,income
74,136620,Anapa,45.002102,37.347301,Belgorod,50.643799,36.590099,2017-02-17T09:25:00Z,2017-02-17T10:15:00Z,2017-02-17 09:30:00,...,629.829866,0.816667,47.101,1800,69238.47,4638.67,12462.92,1577.15,105116.558,420466.23
77,136642,Anapa,45.002102,37.347301,Belgorod,50.643799,36.590099,2017-01-30T09:25:00Z,2017-01-30T10:15:00Z,2017-01-30 09:28:00,...,629.829866,0.816667,47.101,1800,69238.47,4638.67,12462.92,1577.15,88616.558,354466.23
78,136645,Anapa,45.002102,37.347301,Belgorod,50.643799,36.590099,2017-01-05T09:25:00Z,2017-01-05T10:15:00Z,2017-01-05 09:29:00,...,629.829866,0.850000,47.101,1800,72064.53,4828.00,12971.62,1641.52,108298.866,433195.46
86,136678,Anapa,45.002102,37.347301,Belgorod,50.643799,36.590099,2017-01-28T09:25:00Z,2017-01-28T10:15:00Z,2017-01-28 09:27:00,...,629.829866,0.833333,47.101,1800,70651.50,4733.33,12717.27,1609.33,108297.714,433190.86
102,136807,Anapa,45.002102,37.347301,Belgorod,50.643799,36.590099,2017-02-23T09:25:00Z,2017-02-23T10:15:00Z,2017-02-23 09:28:00,...,629.829866,0.833333,47.101,1800,70651.50,4733.33,12717.27,1609.33,88257.714,353030.86
105,136823,Anapa,45.002102,37.347301,Belgorod,50.643799,36.590099,2017-02-05T09:25:00Z,2017-02-05T10:15:00Z,2017-02-05 09:27:00,...,629.829866,0.850000,47.101,1800,72064.53,4828.00,12971.62,1641.52,105778.866,423115.46
109,136844,Anapa,45.002102,37.347301,Belgorod,50.643799,36.590099,2017-02-28T09:25:00Z,2017-02-28T10:15:00Z,2017-02-28 09:26:00,...,629.829866,0.833333,47.101,1800,70651.50,4733.33,12717.27,1609.33,97077.714,388310.86
116,136887,Anapa,45.002102,37.347301,Belgorod,50.643799,36.590099,2017-01-20T09:25:00Z,2017-01-20T10:15:00Z,2017-01-20 09:30:00,...,629.829866,0.816667,47.101,1800,69238.47,4638.67,12462.92,1577.15,101456.558,405826.23
117,136888,Anapa,45.002102,37.347301,Belgorod,50.643799,36.590099,2017-02-14T09:25:00Z,2017-02-14T10:15:00Z,2017-02-14 09:29:00,...,629.829866,0.833333,47.101,1800,70651.50,4733.33,12717.27,1609.33,107277.714,429110.86
120,136922,Anapa,45.002102,37.347301,Belgorod,50.643799,36.590099,2017-02-11T09:25:00Z,2017-02-11T10:15:00Z,2017-02-11 09:27:00,...,629.829866,0.833333,47.101,1800,70651.50,4733.33,12717.27,1609.33,103617.714,414470.86


# Как мы видим не все рейсы заполняются полностью, что приводит к снижению прибыли. Попробуем провести перераспределение, чтобы уменьшить количество рейсов

In [338]:
#здесь будет код, который рассмотрит рейсы в декабре и отдельно в январе, феврале
# чтобы выяснить возможность уменьшить количество рейсов

In [368]:
for i in data.arrival_city.unique():
    sample = data[(data['arrival_city'] == i) & (data['actual_departure'] < datetime.strptime('2017-03-01 00:00', '%Y-%m-%d %H:%M'))]
    print(f'Процент незаполненных мест в январе-феврале 2017 ({i}): {(sample.flight_seats.sum() / sample.aircraft_seats.sum())*100} %')
    print(f'Количество незаполненных мест в самолетах в январе-феврале 2017 года ({i}): {sample.aircraft_seats.sum() - sample.flight_seats.sum()} ')

Процент незаполненных мест в январе-феврале 2017 (Moscow): 87.01434159061277 %
Количество незаполненных мест в самолетах в январе-феврале 2017 года (Moscow): 996 
Процент незаполненных мест в январе-феврале 2017 (Novokuznetsk): 0.0 %
Количество незаполненных мест в самолетах в январе-феврале 2017 года (Novokuznetsk): 1170 
Процент незаполненных мест в январе-феврале 2017 (Belgorod): 92.97571203914032 %
Количество незаполненных мест в самолетах в январе-феврале 2017 года (Belgorod): 402 


# Мы видим, что рейсы не полностью заполняются и мы можем приблизить к друг другу показатели спроса и предложения, чтобы использовать освободившиеся самолеты на более востребованные направления.